In [2]:
pip show azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ModuleNotFoundError: No module named 'azure'

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [16]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
df = pd.read_csv('dataset/breast-cancer.data')

Loading Data...


In [43]:
df

,recurence,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no
...,...,...,...,...,...,...,...,...,...,...
281,recurrence-events,30-39,premeno,30-34,0-2,no,2,left,left_up,no
282,recurrence-events,30-39,premeno,20-24,0-2,no,3,left,left_up,yes
283,recurrence-events,60-69,ge40,20-24,0-2,no,1,right,left_up,no
284,recurrence-events,40-49,ge40,30-34,3-5,no,3,left,left_low,no


In [35]:
df = df[df["node-caps"] != "?"]
df = df[df["breast-quad"] != "?"]

X = df.drop("recurence", axis=1)
X["age"] = X["age"].apply(lambda x: int(x[0]))
X["menopause"] = X["menopause"].apply(lambda x: 1 if x == "lt40" else (2 if x == "ge40" else 3))
X["tumor-size"] = X["tumor-size"].apply(lambda x: int(x[0]))
X["inv-nodes"] = X["inv-nodes"].apply(lambda x: int(x[0]))
X["node-caps"] = X["node-caps"].apply(lambda x: 1 if x == "yes" else 0)
X["breast"] = X["breast"].apply(lambda x: 1 if x == "left" else 0)
X = pd.get_dummies(X, columns=["breast-quad"])
X["irradiat"] = X["irradiat"].apply(lambda x: 1 if x == "yes" else 0)

y = df["recurence"]
y= y.apply(lambda x: 1 if x == "recurrence-events" else 0)

In [37]:
X

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,irradiat,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up
0,3,3,3,0,0,3,1,0,False,True,False,False,False
1,4,3,2,0,0,2,0,0,False,False,False,False,True
2,4,3,2,0,0,2,1,0,False,True,False,False,False
3,6,2,1,0,0,2,0,0,False,False,True,False,False
4,4,3,0,0,0,2,0,0,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,3,3,3,0,0,2,1,0,False,False,True,False,False
282,3,3,2,0,0,3,1,1,False,False,True,False,False
283,6,2,2,0,0,1,0,0,False,False,True,False,False
284,4,2,3,3,0,3,1,0,False,True,False,False,False


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
#train a logistic regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# predict the test set
y_pred = logreg.predict(X_test)

# get the confusionmatrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[53,  3],
       [19,  9]])

In [45]:
# get the roc_auc_score
y_pred_proba = rf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_proba)



np.float64(0.6581632653061225)

In [46]:
# dget the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7380952380952381